In [1]:
# make display window wider
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

#### Simple GUI for evaluating image quality

In [2]:
import os
import csv
import openslide
import tkinter as tk
from tkinter import StringVar, filedialog, messagebox
from PIL import ImageTk, Image

In [3]:
class EvalGui:
    def __init__(self):
        self.gui_setup()
        self.index = None
        self.database = []
    
    def gui_setup(self):
        self.root = tk.Tk()
        self.root.title("Image Quality Evaluation")
        self.root.state("iconic")
        
        self.w = self.root.winfo_screenwidth()
        self.h = self.root.winfo_screenheight()
        self.f = 0.8
        
        self.canvas = tk.Canvas(self.root, width=self.w*self.f, height=self.h)
        self.canvas.grid(row=0, column=0)
        
        self.control = tk.Frame(self.root, width=self.w*(1-self.f), height=self.h)
        self.control.grid(row=0, column=1)
        
        self.open_d = tk.Button(self.control, text="open dir", command=self.load_files)
        self.open_d.grid(row=0, column=0, sticky=tk.EW, ipady=5, padx=5, pady=10)
        
        self.save_d = tk.Button(self.control, text="set eval save dir", command=self.set_save)
        self.save_d.grid(row=1, column=0, sticky=tk.EW, ipady=5, padx=5, pady=10)
        
        self.progress = tk.Label(self.control, text="-- / --")
        self.progress.grid(row=2, column=0, sticky=tk.EW, ipady=5, padx=5, pady=10)
        self.filename = tk.Label(self.control, text="xxxxxxxx")
        self.filename.grid(row=3, column=0, sticky=tk.EW, ipady=5, padx=5, pady=10)
        
        self.previous = tk.Button(self.control, text="previous", command=lambda: self.update(step=-1))
        self.previous.grid(row=4, column=0, sticky=tk.EW, ipady=5, padx=5, pady=10)
        self.next = tk.Button(self.control, text="next", command=lambda: self.update(step=1))
        self.next.grid(row=5, column=0, sticky=tk.EW, ipady=5, padx=5, pady=10)
        
        self.radiobox = StringVar()
        self.good = tk.Radiobutton(self.control, text="good", value="good", variable=self.radiobox)
        self.good.grid(row=6, column=0, sticky=tk.W, ipady=5, padx=5, pady=5)
        self.medium = tk.Radiobutton(self.control, text="medium", value="medium", variable=self.radiobox)
        self.medium.grid(row=7, column=0, sticky=tk.W, ipady=5, padx=5, pady=5)
        self.bad = tk.Radiobutton(self.control, text="bad", value="bad", variable=self.radiobox)
        self.bad.grid(row=8, column=0, sticky=tk.W, ipady=5, padx=5, pady=5)
        
        self.note = tk.Entry(self.control)
        self.note.grid(row=9, column=0, sticky=tk.EW, ipady=5, padx=5, pady=10)
        
        self.confirm = tk.Button(self.control, text="confirm", command=self.save_eval)
        self.confirm.grid(row=10, column=0, sticky=tk.EW, ipady=5, padx=5, pady=10)
        self.confirm_next = tk.Button(self.control, text="confirm and next", command=self.save_and_next)
        self.confirm_next.grid(row=11, column=0, sticky=tk.EW, ipady=5, padx=5, pady=10)
        
    def load_files(self):
        file_dir = filedialog.askdirectory()
        if not file_dir:
            messagebox.showinfo("warning", "no directory choosed")
        else:
            self.index = None
            del self.database
            self.database = []
            fnames = os.listdir(file_dir)
            fnames.sort()
            for fname in fnames:
                if fname.endswith(".kfb") or fname.endswith(".tif"):
                    self.database.append({"fname":os.path.join(file_dir, fname), "image":None})
            if not self.database:
                messagebox.showinfo("warning", "no kfb/tif file exists")
            else:
                self.index = 0
                self.evaluation = os.path.join(file_dir, "evaluation.csv")
                self.update()
    
    def set_save(self):
        file_dir = filedialog.askdirectory()
        if not file_dir:
            messagebox.showinfo("warning", "no directory choosed")
        else:
            self.evaluation = os.path.join(file_dir, "evaluation.csv")
    
    def update_text(self):
        self.progress.config(text="{}: {} / {}".format(os.path.basename(os.path.dirname(self.database[self.index]["fname"])),
                                                       self.index+1, len(self.database)))
        self.filename.config(text=os.path.basename(self.database[self.index]["fname"]))
    
    def load_image(self):
        def resize(image, w, h):
            w0, h0 = image.size
            scale = min(w/w0, h/w0)
            return image.resize((int(w0*scale), int(h0*scale)))
        
        if self.database[self.index]["image"] is not None:
            image = self.database[self.index]["image"]
        else:
            slide = openslide.OpenSlide(self.database[self.index]["fname"])
            level_count = slide.level_count
            m_top, n_top = slide.level_dimensions[level_count-1]
            image = slide.get_thumbnail((m_top, n_top))
            image = ImageTk.PhotoImage(resize(image, self.w*self.f, self.h))
            self.database[self.index]["image"] = image
        self.canvas.delete("all")
        self.canvas.create_image(self.w*self.f/2, self.h/2, image=image)
        
    def update(self, step=0):
        if self.index == None:
            messagebox.showinfo("error", "three is no kfb/tif file loaded")
            return
        self.index += step
        if self.index not in range(len(self.database)):
            messagebox.showinfo("warning", "already the end")
            self.index -= step
            return
        self.update_text()
        self.load_image()
        
    def save_eval(self):
        if self.index == None:
            messagebox.showinfo("error", "three is no kfb/tif file loaded")
            return
        # get evaluation
        choice = self.radiobox.get()
        note = self.note.get()
        self.note.delete(0, tk.END)
        # save evaluation to csv file
        isfile = os.path.isfile(self.evaluation)
        with open(self.evaluation, 'a') as csv_file:
            csv_writer = csv.writer(csv_file, delimiter=',')
            if not isfile:
                csv_writer.writerow(["folder", "filename", "evaluation", "note"])
            csv_writer.writerow([os.path.basename(os.path.dirname(self.database[self.index]["fname"])), 
                                 os.path.basename(self.database[self.index]["fname"]), 
                                 choice, 
                                 note])
            
    def save_and_next(self):
        self.save_eval()
        self.update(step=1)
        
    def run(self):
        self.root.mainloop()
        

In [4]:
EvalGui().run()

#### Analyze image quality evaluations

In [6]:
import numpy as np
import pandas as pd

In [7]:
eval_file = '/media/tsimage001/Elements/data/asap_all/evaluation.csv'
df = pd.read_csv(eval_file)
df.sample(5)

,folder,filename,evaluation,note
155,01_ASCUS,2018-03-14-22_20_38.tif,good,NaN
226,01_ASCUS,2018-03-15-12_28_06.tif,good,NaN
759,02_LSIL,2017-10-19-09_21_43.tif,good,NaN
474,01_ASCUS,2018-03-19-14_45_35.tif,good,NaN
486,01_ASCUS,2018-03-20-18_23_58.tif,good,NaN


In [12]:
classes = pd.unique(df.folder).tolist()
print(classes)

['01_ASCUS', '02_LSIL', '03_ASCH', '04_HSIL', '05_SCC', '06_AGC1', '07_AGC2', '08_ADC', '09_EC', '10_FUNGI', '11_TRI', '12_CC', '13_ACTINO', '14_VIRUS']


In [24]:
# count the occurance of good/medium/bad in each class
rows = []
for class_i in classes:
    row = {}
    row["folder"] = class_i
    row["total"] = df[df.folder == class_i].shape[0]
    row["good"] = df[(df.folder == class_i) & (df.evaluation == "good")].shape[0]
    row["not good (medium/bad)"] = df[(df.folder == class_i) & (df.evaluation != "good")].shape[0]
    rows.append(row)

df_sum = pd.DataFrame(rows)
df_sum.head(5)

,folder,good,not good (medium/bad),total
0,01_ASCUS,457,48,505
1,02_LSIL,301,26,327
2,03_ASCH,46,9,55
3,04_HSIL,96,15,111
4,05_SCC,31,9,40


In [25]:
# write to csv
df_sum.to_csv('/media/tsimage001/Elements/data/asap_all/evaluation_.csv')

#### Show some sample images

In [35]:
import os
import openslide
import cv2
import numpy as np
import pandas as pd

In [36]:
eval_file = '/media/tsimage001/Elements/data/asap_all/evaluation.csv'
tif_folder = '/media/tsimage001/Elements/data/asap_all'
df = pd.read_csv(eval_file)
df.sample(3)

,folder,filename,evaluation,note,Unnamed: 4,summary,folder.1,good,not good (medium/bad),total
108,01_ASCUS,2018-03-14-20_35_59.tif,good,NaN,NaN,NaN,NaN,NaN,NaN,NaN
292,01_ASCUS,2018-03-15-15_20_15.tif,good,NaN,NaN,NaN,NaN,NaN,NaN,NaN
515,02_LSIL,2017-10-09-16_53_20.tif,good,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
def get_thumbnail(tif_name):
    slide = openslide.OpenSlide(tif_name)
    level_count = slide.level_count
    m_top, n_top = slide.level_dimensions[level_count-1]
    image = slide.get_thumbnail((m_top, n_top))
    return image

def save_thumbnail(df, save_path):
    os.makedirs(save_path, exist_ok=True)
    for _,row in df.iterrows():
        tif_name = os.path.join(tif_folder, row["folder"], row["filename"])
        image = get_thumbnail(tif_name)
        image = np.asarray(image)
        cv2.putText(image, row["evaluation"]+": "+row["note"], (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,0))
        image_name = os.path.join(save_path, os.path.splitext(row["filename"])[0]+".jpg")
        cv2.imwrite(image_name, image)

In [58]:
save_path = '/media/tsimage001/Elements/data/asap_all/bad_samples'
df_medium = df[df.evaluation == "medium"]
df_m_sam = df_medium.sample(20)

df_bad = df[df.evaluation == "bad"]
df_b_sam = df_bad.sample(10)

In [59]:
save_thumbnail(df_m_sam, save_path)
save_thumbnail(df_b_sam, save_path)